##This notebook explores generating Amazon product descriptions using different large language models (LLMs) and prompting techniques (zero-shot and few-shot).

Key steps and findings:

1.  Prompt Engineering:
    *   Defined functions to create zero-shot and few-shot prompts for product description generation.
2.  FLAN-T5 Model:
    *   Used the google/flan-t5-base model for text generation.
    *   Generated product descriptions using both zero-shot and few-shot prompting with FLAN-T5.
    *   Evaluated the generated descriptions using metrics like BLEU, ROUGE, BERTScore, and Flesch Reading Ease.

3.  OpenAI GPT-3.5 Model:
    *   Used the GPT-3.5-turbo model.
    *   Generated product descriptions using both zero-shot and few-shot prompting with GPT-3.5.
    *   Evaluated the generated descriptions using the same metrics.

4.  Gemini 1.5 Pro Model:
    *   Used the the Gemini 1.5 Pro model.
    *   Generated product descriptions using both zero-shot and few-shot prompting with Gemini 1.5 Pro.
    *   Evaluated the generated descriptions using the same metrics.

## Generating Descriptions with LLMs

Defining functions to create zero-shot and few-shot prompts for product description generation. The few-shot prompt includes two examples to guide the model.

In [ ]:
# Zero-shot prompt builder
def build_zero_shot_prompt(row):
    return (
        "Generate a high-quality Amazon product description using the following product metadata:\n\n"
        f"{row['input_text']}\n\n"
        "Description:"
    )


# Few-shot prompt builder
def build_few_shot_prompt(row):
    # Example few-shot prompt with 2 examples
    examples = """
Example 1:
TITLE: Wireless Bluetooth Headphones
BULLET_POINTS: High-quality sound, noise cancellation, 20 hours battery life
Description: These Wireless Bluetooth Headphones offer high-quality sound with noise cancellation and a long-lasting 20-hour battery life.

Example 2:
TITLE: Stainless Steel Water Bottle
BULLET_POINTS: 1 liter capacity, keeps drinks cold for 24 hours, BPA free
Description: This Stainless Steel Water Bottle has a 1-liter capacity, keeps drinks cold for 24 hours, and is BPA free.

Now generate a description for the following product metadata:

""" + row['input_text'] + "\n\nDescription:"
    return examples

In [ ]:
from transformers import pipeline

flan_t5_generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device_map="auto",
    max_new_tokens=150
)

def generate_description_flan_zero(row):
    prompt = build_zero_shot_prompt(row)
    try:
        result = flan_t5_generator(prompt)
        return result[0]['generated_text'].strip()
    except Exception as e:
        print("FLAN-T5 zero-shot error:", e)
        return None

def generate_description_flan_few(row):
    prompt = build_few_shot_prompt(row)
    try:
        result = flan_t5_generator(prompt)
        return result[0]['generated_text'].strip()
    except Exception as e:
        print("FLAN-T5 few-shot error:", e)
        return None


# Generate zero-shot descriptions
df_llm_input['generated_flan_zero'] = df_llm_input.apply(generate_description_flan_zero, axis=1)
# Generate few-shot descriptions
df_llm_input['generated_flan_few'] = df_llm_input.apply(generate_description_flan_few, axis=1)


# Evaluate
results_flan_zero = evaluate_model(df_llm_input, gen_col="generated_flan_zero", ref_col="target_description")
results_flan_few = evaluate_model(df_llm_input, gen_col="generated_flan_few", ref_col="target_description")

print("Zero-shot results:", results_flan_zero)
print("Few-shot results:", results_flan_few)

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Evaluation for Model: generated_flan_zero
BLEU Score         : 0.0715
ROUGE-1 Score      : 0.2447
ROUGE-2 Score      : 0.0845
BERTScore-F1       : 0.8247
Flesch Reading Ease: 33.54

Evaluation for Model: generated_flan_few
BLEU Score         : 0.0795
ROUGE-1 Score      : 0.2608
ROUGE-2 Score      : 0.1134
BERTScore-F1       : 0.8323
Flesch Reading Ease: 35.02
Zero-shot results: {'BLEU': 0.07148332948405778, 'ROUGE-1': np.float64(0.24466993174474774), 'ROUGE-2': np.float64(0.0845171347106389), 'BERTScore-F1': np.float64(0.8247116899490357), 'Flesch': np.float64(33.53942933172864)}
Few-shot results: {'BLEU': 0.07952200700666757, 'ROUGE-1': np.float64(0.26079258828716323), 'ROUGE-2': np.float64(0.11336626963227353), 'BERTScore-F1': np.float64(0.8323147225379944), 'Flesch': np.float64(35.01568689983381)}


In [ ]:
df_llm_input[['input_text', 'generated_flan_zero','generated_flan_few']]

,input_text,generated_flan_zero,generated_flan_few
0,"TITLE: Plane Light System, Plastic + Metal Tax...",[Full set of bright LED lights for your model ...,[Full set of bright LED lights for your model ...
1,TITLE: DECOR Kafe Home Decor Sunflower Wall St...,"DECOR Kafe Home Decor Sunflower Wall Sticker, ...","DECOR Kafe Home Decor Sunflower Wall Sticker, ..."
2,TITLE: Vbuyz Women's Rayon Foil Print Stitched...,Vbuyz Women's Rayon Foil Print Stitched Straig...,Vbuyz Women's Rayon Foil Print Stitched Straig...
3,TITLE: Mitsui Shop on Suruga Street in Edo by ...,Framed Canvas Wall Art Decor | Fine Artwork Pa...,Mitsui Shop on Suruga Street in Edo by Katsush...
4,"TITLE: Brass Glass ( 1 pcs ), 250ml (Glass wit...","[Specification:-Set fo 1, Material: Brass, Vol...","[Specification:-Set fo 1, Material: Brass, Vol..."
5,"TITLE: EcoSmart PolyPaper Cutting Board, White...",[EcoSmart PolyPaper Cutting Boards are durable...,[EcoSmart PolyPaper Cutting Boards are durable...
6,TITLE: Genuine Mazda Parts BBM6-56-620 Hood La...,This is the official Genuine Mazda Parts repla...,The Genuine Mazda Parts BBM6-56-620 Hood Latch...
7,TITLE: Candere by Kalyan Jewellers BIS Hallmar...,[These gold rings for women are set in 14kt (5...,[These gold rings for women are set in 14kt (5...
8,"TITLE: Mini 360° Panoramic Tripod Ball Head, 1...","Mini 360° Panoramic Tripod Ball Head, 1/4"" Scr...","Mini 360° Panoramic Tripod Ball Head, 1/4"" Scr..."
9,TITLE: Habeeb Bags Boy's and Girl's 13 Inches ...,"[Stylish and light weight , durable canvas lin...",Habeeb Bags Boy's and Girl's 13 Inches Leather...


In [ ]:
# original vs generated descriptions
for i in range(2):
    print(f"Product Title:\n{df_llm_input.loc[i, 'title']}")
    print(f"\n* Original Description:\n{df_llm_input.loc[i, 'target_description']}")
    print(f"\n🤖 Generated Desc by FLAN T5 (zero-shot):\n{df_llm_input.loc[i, 'generated_flan_zero']}")
    print(f"\n🤖 Generated Desc by FLAN T5 (few-shot):\n{df_llm_input.loc[i, 'generated_flan_few']}")
    print('---' * 20)

Product Title:
Plane Light System, Plastic + Metal Taxi Lights Airplane LED Light, for Model Plane

* Original Description:
Features:&nbsp;<br> Full set of bright LED lights for your model plane or glider. <br> Use the power supplied by the receiver and don't require its own battery. <br> The circuitry has been specially designed so that the landing lights and taxi lights can be switched on and off directly. <br> Either use a Y-harness and link it to another channel such as the gear or assign a separate channel to it. <br> It includes red beacon light, white taxi light, white strobe light, white landing light, green navigation light, red navigation light. <br> <br>Specification:&nbsp;<br>Material: Plastic + Metal<br>Color: Shown As Pictures<br>Weight: 111g<br>Type: RC Part &amp; Accessory<br>Control Board Size: Approx. 54 * 35 * 13mm / 2.1 * 1.4 * 0.5inch<br> Cable Size: Approx. OD 0.8mm*0.8m <br> LED Quantity: 14pcs <br>Lighting System Specification:<br>2 x Red Beacon Lights (#11, #12

**Analysis for FLAN-T5:**

This model performed reasonably well, particularly with few-shot prompting, achieving the high BLEU and ROUGE-2 scores. This suggests it was good at generating descriptions with similar phrasing to the original text. However, its Flesch Reading Ease scores was low, indicating that its generated text might be harder to read.

In [ ]:
!pip install openai
import openai
from openai import OpenAI
import os

client = OpenAI(
    api_key="removed key for security purposes"
)

# GPT-3.5 Zero-Shot
def generate_description_openai_zero(row):
    prompt = build_zero_shot_prompt(row)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Generate Amazon product descriptions from given metadata."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("OpenAI zero-shot error:", e)
        return None

# GPT-3.5 Few-Shot
def generate_description_openai_few(row):
    prompt = build_few_shot_prompt(row)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Generate Amazon product descriptions from given metadata."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("OpenAI few-shot error:", e)
        return None


# Apply generation
df_llm_input['generated_openai_zero'] = df_llm_input.apply(generate_description_openai_zero, axis=1)
df_llm_input['generated_openai_few'] = df_llm_input.apply(generate_description_openai_few, axis=1)

# Evaluate
results_openai_zero = evaluate_model(df_llm_input, gen_col="generated_openai_zero", ref_col="target_description")
results_openai_few = evaluate_model(df_llm_input, gen_col="generated_openai_few", ref_col="target_description")

# Output
print("OpenAI GPT-3.5 Zero-shot:", results_openai_zero)
print("OpenAI GPT-3.5 Few-shot:", results_openai_few)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Evaluation for Model: generated_openai_zero
BLEU Score         : 0.0464
ROUGE-1 Score      : 0.2635
ROUGE-2 Score      : 0.0704
BERTScore-F1       : 0.8356
Flesch Reading Ease: 47.72

Evaluation for Model: generated_openai_few
BLEU Score         : 0.0634
ROUGE-1 Score      : 0.2745
ROUGE-2 Score      : 0.0845
BERTScore-F1       : 0.8407
Flesch Reading Ease: 50.18
OpenAI GPT-3.5 Zero-shot: {'BLEU': 0.04635798651143328, 'ROUGE-1': np.float64(0.263486430007127), 'ROUGE-2': np.float64(0.07036129160463366), 'BERTScore-F1': np.float64(0.8355629920959473), 'Flesch': np.float64(47.723125544062334)}
OpenAI GPT-3.5 Few-shot: {'BLEU': 0.06339489236628446, 'ROUGE-1': np.float64(0.27447922933536584), 'ROUGE-2': np.float64(0.08448765094004426), 'BERTScore-F1': np.float64(0.8407469964027405), 'Flesch': np.float64(50.17769237884623)}


In [ ]:
# original vs generated descriptions
for i in range(2):
    print(f"Product Title:\n{df_llm_input.loc[i, 'title']}")
    print(f"\n* Original Description:\n{df_llm_input.loc[i, 'target_description']}")
    print(f"\n🤖 Generated Desc by gpt 3.5-turbo (zero-shot):\n{df_llm_input.loc[i, 'generated_openai_zero']}")
    print(f"\n🤖 Generated Desc by gpt 3.5-turbo (few-shot):\n{df_llm_input.loc[i, 'generated_openai_few']}")
    print('---' * 20)

Product Title:
Plane Light System, Plastic + Metal Taxi Lights Airplane LED Light, for Model Plane

* Original Description:
Features:&nbsp;<br> Full set of bright LED lights for your model plane or glider. <br> Use the power supplied by the receiver and don't require its own battery. <br> The circuitry has been specially designed so that the landing lights and taxi lights can be switched on and off directly. <br> Either use a Y-harness and link it to another channel such as the gear or assign a separate channel to it. <br> It includes red beacon light, white taxi light, white strobe light, white landing light, green navigation light, red navigation light. <br> <br>Specification:&nbsp;<br>Material: Plastic + Metal<br>Color: Shown As Pictures<br>Weight: 111g<br>Type: RC Part &amp; Accessory<br>Control Board Size: Approx. 54 * 35 * 13mm / 2.1 * 1.4 * 0.5inch<br> Cable Size: Approx. OD 0.8mm*0.8m <br> LED Quantity: 14pcs <br>Lighting System Specification:<br>2 x Red Beacon Lights (#11, #12

**Analysis for GPT-3.5:**


This model, especially with few-shot prompting, demonstrated strong performance across several metrics, including high ROUGE-1 and BERTScore-F1 scores. It also produced descriptions with relatively high readability (Flesch Reading Ease). The example outputs showed that GPT-3.5 generated more detailed and human-like descriptions compared to FLAN-T5.

In [ ]:
!pip install google-generativeai
import google.generativeai as genai

genai.configure(api_key="removed key for security purposes")
model = genai.GenerativeModel("gemini-1.5-pro")

# Gemini Zero-Shot
def generate_description_gemini_zero(row):
    prompt = build_zero_shot_prompt(row)
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print("Gemini zero-shot error:", e)
        return None

# Gemini Few-Shot
def generate_description_gemini_few(row):
    prompt = build_few_shot_prompt(row)
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print("Gemini few-shot error:", e)
        return None


# Apply Gemini generation
df_llm_input['generated_gemini_zero'] = df_llm_input.apply(generate_description_gemini_zero, axis=1)
df_llm_input['generated_gemini_few'] = df_llm_input.apply(generate_description_gemini_few, axis=1)

# Evaluate
results_gemini_zero = evaluate_model(df_llm_input, gen_col="generated_gemini_zero", ref_col="target_description")
results_gemini_few = evaluate_model(df_llm_input, gen_col="generated_gemini_few", ref_col="target_description")

# Print
print("Gemini Zero-shot:", results_gemini_zero)
print("Gemini Few-shot:", results_gemini_few)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Evaluation for Model: generated_gemini_zero
BLEU Score         : 0.0199
ROUGE-1 Score      : 0.2086
ROUGE-2 Score      : 0.0531
BERTScore-F1       : 0.8110
Flesch Reading Ease: 43.47

Evaluation for Model: generated_gemini_few
BLEU Score         : 0.0402
ROUGE-1 Score      : 0.2640
ROUGE-2 Score      : 0.0747
BERTScore-F1       : 0.8367
Flesch Reading Ease: 54.60
Gemini Zero-shot: {'BLEU': 0.019895550649841044, 'ROUGE-1': np.float64(0.2085650886205057), 'ROUGE-2': np.float64(0.05310525112764493), 'BERTScore-F1': np.float64(0.8109968018531799), 'Flesch': np.float64(43.46576193332608)}
Gemini Few-shot: {'BLEU': 0.04015248373902239, 'ROUGE-1': np.float64(0.26398071488524555), 'ROUGE-2': np.float64(0.07466238959260951), 'BERTScore-F1': np.float64(0.8367015552520752), 'Flesch': np.float64(54.59649022808031)}


In [ ]:
# original vs generated descriptions
for i in range(2):
    print(f"Product Title:\n{df_llm_input.loc[i, 'title']}")
    print(f"\n* Original Description:\n{df_llm_input.loc[i, 'target_description']}")
    print(f"\n🤖 Generated Desc by Gemini Pro (zero-shot):\n{df_llm_input.loc[i, 'generated_gemini_zero']}")
    print(f"\n🤖 Generated Desc by Gemini Pro (few-shot):\n{df_llm_input.loc[i, 'generated_gemini_few']}")
    print('---' * 20)

Product Title:
Plane Light System, Plastic + Metal Taxi Lights Airplane LED Light, for Model Plane

* Original Description:
Features:&nbsp;<br> Full set of bright LED lights for your model plane or glider. <br> Use the power supplied by the receiver and don't require its own battery. <br> The circuitry has been specially designed so that the landing lights and taxi lights can be switched on and off directly. <br> Either use a Y-harness and link it to another channel such as the gear or assign a separate channel to it. <br> It includes red beacon light, white taxi light, white strobe light, white landing light, green navigation light, red navigation light. <br> <br>Specification:&nbsp;<br>Material: Plastic + Metal<br>Color: Shown As Pictures<br>Weight: 111g<br>Type: RC Part &amp; Accessory<br>Control Board Size: Approx. 54 * 35 * 13mm / 2.1 * 1.4 * 0.5inch<br> Cable Size: Approx. OD 0.8mm*0.8m <br> LED Quantity: 14pcs <br>Lighting System Specification:<br>2 x Red Beacon Lights (#11, #12

**Analysis of Gemini 1.5 Pro:**

While its BLEU and ROUGE-2 scores were low in the few-shot setting, Gemini 1.5 Pro achieved a high Flesch Reading Ease score, suggesting it generated the most readable descriptions. The example outputs showed that Gemini 1.5 Pro also produced detailed and well-structured descriptions.